In [27]:
import numpy as np
import pandas as pd
import os
import yaml

pd.options.display.max_rows = 999

#### Paths & Vars

In [1]:
# vars
adapt_methods = ['coral','al_layer']
gen_methods = ['irm','dro','coral','al_layer']
tasks = ['mortality','longlos','invasivevent','sepsis']
metrics = ['auc','auprc','ace_abs_logistic_log']
Ns = [100,500,1000,1500]

# paths
tables_fpath = '/hpf/projects/lsung/projects/mimic4ds/Experiments/domain_adapt/tables/'
adapt_fpath = '/hpf/projects/lsung/projects/mimic4ds/Experiments/domain_adapt'
gen_fpath = '/hpf/projects/lsung/projects/mimic4ds/Experiments/domain_gen'
base_fpath = '/hpf/projects/lsung/projects/mimic4ds/Experiments/baseline'

In [53]:
alpha = 0.05

results = {
    "base":{
        "eval":pd.read_csv(f"{base_fpath}/results/model_evaluation_by_gender_{alpha}.csv").query("phase=='test'"),
        "compare":pd.read_csv(f"{base_fpath}/results/model_comparison_by_gender_{alpha}.csv").query("phase=='test'"),
    },
    "gen":{
        "eval":pd.read_csv(f"{gen_fpath}/results/model_evaluation_by_gender_{alpha}.csv").query("phase=='test' and group==1"),
        "compare":pd.read_csv(f"{gen_fpath}/results/model_comparison_by_gender_{alpha}.csv").query("phase=='test' and group==1"),
    },
    "adapt":{
        "eval":pd.read_csv(f"{adapt_fpath}/results/model_evaluation_by_gender_{alpha}.csv").query("phase=='test' and group==1"),
        "compare":pd.read_csv(f"{adapt_fpath}/results/model_comparison_by_gender_{alpha}.csv").query("phase=='test' and group==1"),
    },
}

df_results_all = {}
for gender in ['F','M']:
    # base results
    df_base = results['base']['eval'].query(
        "\
        train_group=='2008 - 2010' and\
        eval_group=='2017 - 2019' and\
        metric==@metrics and\
        gender==@gender\
        "
    )[['analysis_id', 'metric', 'ci_lower','ci_med','ci_upper']]
    df_base['group']=1
    df_base['train_method']='Baseline [08-10]'

    # oracle results
    df_oracle = results['base']['eval'].query(
        "\
        train_group=='2017 - 2019' and\
        eval_group=='2017 - 2019' and\
        metric==@metrics and\
        gender==@gender\
        "
    )[['analysis_id', 'metric', 'ci_lower','ci_med','ci_upper']]
    df_oracle['group']=1
    df_oracle['train_method'] = 'Oracle [17-19]'

    # ERM results
    df_erm = results['gen']['eval'].query(
        "\
        train_method=='erm' and\
        metric==@metrics and\
        gender==@gender\
        "
    )[['analysis_id', 'metric', 'ci_lower','ci_med','ci_upper','group','train_method']]

    # domain gen results
    df_gen = results['gen']['eval'].query(
        "\
        train_method!='erm' and\
        metric==@metrics and\
        gender==@gender\
        "
    )[['analysis_id', 'metric', 'ci_lower','ci_med','ci_upper','group','train_method']]
    df_gen['framework'] = 'Domain Generalization'

    stats = results['gen']['compare'].query(
        "\
        metric==@metrics and\
        gender==@gender\
        "
    )[['analysis_id', 'metric', 'ci_lower','ci_upper','group','train_method']]
    stats['sig'] = stats['ci_lower']*stats['ci_upper']>0
    stats.drop(columns=['ci_lower','ci_upper'],inplace=True)

    df_gen = pd.merge(
        df_gen, 
        stats, 
        how='left', 
        left_on=['analysis_id','metric','group', 'train_method'],
        right_on = ['analysis_id','metric','group','train_method']
    )

    # domain adapt results
    df_adapt = results['adapt']['eval'].query(
        "\
        metric==@metrics and\
        gender==@gender\
        "
    )[['analysis_id', 'metric', 'ci_lower','ci_med','ci_upper','group','train_method','n_ood']]
    df_adapt['framework'] = 'Domain Adaptation'

    stats = results['adapt']['compare'].query(
        "\
        metric==@metrics and\
        gender==@gender\
        "
    )[['analysis_id', 'metric', 'ci_lower','ci_upper','group','train_method','n_ood']]
    stats['sig'] = stats['ci_lower']*stats['ci_upper']>0
    stats.drop(columns=['ci_lower','ci_upper'],inplace=True)

    df_adapt = pd.merge(
        df_adapt, 
        stats, 
        how='left', 
        left_on=['analysis_id','metric','group', 'train_method','n_ood'],
        right_on = ['analysis_id','metric','group','train_method','n_ood']
    )

    # join results
    df_results = pd.concat((df_base, df_oracle, df_erm, df_gen, df_adapt))
    df_results['sig'].replace({
        False:'',
        True:'*',
        np.nan:''
    }, inplace=True)

    # combine CI columns into performance column
    df_results['Performance'] = (
        df_results['ci_med'].apply('{:.3f}'.format) + 
        df_results['sig'] +
        " (" + 
        df_results['ci_lower'].apply('{:.3f}'.format) + 
        ',' + 
        df_results['ci_upper'].apply('{:.3f}'.format) +
        ')'
    )
    df_results.drop(columns=['ci_lower','ci_med','ci_upper'],inplace=True)
    df_results.fillna(" ",inplace=True)

    # rename columns and values
    df_results.rename(columns = {
        'metric':'Metric',
        'group':'Year Group',
        'analysis_id':'Task',
        'n_ood':'Unlabeled OOD Samples',
        'framework': 'Framework',
        'train_method':'Method'},inplace=True)

    # Replace values
    df_results['Year Group'].replace(
        {
            0:"2008 - 2016 [ID]",
            1:"2017 - 2019 [OOD]"
        },
        inplace=True
    )
    df_results['Metric'].replace(
        {
            'auc':"AUROC",
            'auprc':"AUPRC",
            'ace_abs_logistic_log':'Calibration'
        },
        inplace=True
    )
    df_results['Method'].replace(
        {
            'al_layer':"AL",
            'coral':"CORAL",
            'erm':"ERM",
            'irm':'IRM',
            'dro':'GroupDRO'
        },
        inplace=True
    )
    df_results['Task'].replace(
        {
            'longlos':'Long LOS',
            'sepsis':'Sepsis',
            'mortality':'Mortality',
            'invasivevent':'Invasive Ventilation',
        },
        inplace=True
    )
    

    # pivot table    
    df_results = df_results.pivot(index=["Task","Year Group","Metric"],columns=["Framework","Unlabeled OOD Samples","Method"],values=["Performance"])
    df_results.fillna(" ",inplace = True)
    df_results.columns = pd.MultiIndex.from_tuples([x[1:] for x in df_results.columns], names = ['Framework','Unlabelled OOD Samples','Method'])

    # Order Indices & Columns
    df_results = df_results[[
        (                    ' ',    ' ', 'Baseline [08-10]'),
        (                    ' ',    ' ',   'Oracle [17-19]'),
        (                    ' ',    ' ',              'ERM'),
        ('Domain Generalization',    ' ',              'IRM'),
        ('Domain Generalization',    ' ',         'GroupDRO'),
        ('Domain Generalization',    ' ',               'AL'),
        ('Domain Generalization',    ' ',            'CORAL'),
        (    'Domain Adaptation',  100.0,               'AL'),
        (    'Domain Adaptation',  100.0,            'CORAL'),
        (    'Domain Adaptation',  500.0,               'AL'),
        (    'Domain Adaptation',  500.0,            'CORAL'),
        (    'Domain Adaptation', 1000.0,               'AL'),
        (    'Domain Adaptation', 1000.0,            'CORAL'),
        (    'Domain Adaptation', 1500.0,               'AL'),
        (    'Domain Adaptation', 1500.0,            'CORAL'),
    ]]

    df_results = df_results.reindex(labels = ['Long LOS','Sepsis','Mortality','Invasive Ventilation'], level=0)
    df_results = df_results.reindex(labels = ['AUROC','AUPRC','Calibration'],level=2)
    #df_results = df_results.style.apply(highlight_sig, axis=0)

    # add to dictionary
    df_results_all[gender] = df_results

In [54]:
df_results_all['F']

Framework                                                                \
Unlabelled OOD Samples                                                    
Method                                                 Baseline [08-10]   
Task                 Year Group        Metric                             
Long LOS             2017 - 2019 [OOD] AUROC        0.614 (0.600,0.627)   
                                       AUPRC        0.479 (0.459,0.499)   
                                       Calibration  0.096 (0.087,0.105)   
Sepsis               2017 - 2019 [OOD] AUROC        0.661 (0.636,0.686)   
                                       AUPRC        0.204 (0.181,0.229)   
                                       Calibration  0.044 (0.037,0.052)   
Mortality            2017 - 2019 [OOD] AUROC        0.863 (0.846,0.879)   
                                       AUPRC        0.484 (0.442,0.523)   
                                       Calibration  0.021 (0.017,0.026)   
Invasive Ventilation 2017 - 2019 [OOD] AUROC        0.872 (0.850,0.892)   
                                       AUPRC        0.676 (0.634,0.716)   
                                       Calibration  0.030 (0.026,0.035)   

Framework                                                                \
Unlabelled OOD Samples                                                    
Method                                                   Oracle [17-19]   
Task                 Year Group        Metric                             
Long LOS             2017 - 2019 [OOD] AUROC        0.702 (0.690,0.714)   
                                       AUPRC        0.534 (0.513,0.554)   
                                       Calibration  0.039 (0.032,0.047)   
Sepsis               2017 - 2019 [OOD] AUROC        0.691 (0.667,0.716)   
                                       AUPRC        0.284 (0.250,0.319)   
                                       Calibration  0.020 (0.015,0.026)   
Mortality            2017 - 2019 [OOD] AUROC        0.887 (0.870,0.901)   
                                       AUPRC        0.490 (0.449,0.531)   
                                       Calibration  0.022 (0.018,0.026)   
Invasive Ventilation 2017 - 2019 [OOD] AUROC        0.870 (0.847,0.891)   
                                       AUPRC        0.674 (0.629,0.715)   
                                       Calibration  0.021 (0.018,0.024)   

Framework                                                                \
Unlabelled OOD Samples                                                    
Method                                                              ERM   
Task                 Year Group        Metric                             
Long LOS             2017 - 2019 [OOD] AUROC        0.651 (0.640,0.661)   
                                       AUPRC        0.501 (0.486,0.516)   
                                       Calibration  0.065 (0.058,0.071)   
Sepsis               2017 - 2019 [OOD] AUROC        0.646 (0.626,0.666)   
                                       AUPRC        0.229 (0.207,0.251)   
                                       Calibration  0.038 (0.032,0.043)   
Mortality            2017 - 2019 [OOD] AUROC        0.892 (0.880,0.904)   
                                       AUPRC        0.557 (0.526,0.588)   
                                       Calibration  0.023 (0.020,0.026)   
Invasive Ventilation 2017 - 2019 [OOD] AUROC        0.877 (0.859,0.893)   
                                       AUPRC        0.692 (0.659,0.722)   
                                       Calibration  0.023 (0.021,0.026)   

Framework                                          Domain Generalization  \
Unlabelled OOD Samples                                                     
Method                                                               IRM   
Task                 Year Group        Metric                              
Long LOS             2017 - 2019 [OOD] AUROC         0.651 (0.641,0.662)   
                 

In [56]:
df_results_all['M']

Framework                                                                \
Unlabelled OOD Samples                                                    
Method                                                 Baseline [08-10]   
Task                 Year Group        Metric                             
Long LOS             2017 - 2019 [OOD] AUROC        0.664 (0.652,0.675)   
                                       AUPRC        0.538 (0.522,0.556)   
                                       Calibration  0.085 (0.077,0.093)   
Sepsis               2017 - 2019 [OOD] AUROC        0.633 (0.614,0.651)   
                                       AUPRC        0.144 (0.131,0.157)   
                                       Calibration  0.081 (0.074,0.087)   
Mortality            2017 - 2019 [OOD] AUROC        0.889 (0.876,0.902)   
                                       AUPRC        0.466 (0.428,0.504)   
                                       Calibration  0.013 (0.011,0.016)   
Invasive Ventilation 2017 - 2019 [OOD] AUROC        0.878 (0.864,0.890)   
                                       AUPRC        0.630 (0.598,0.662)   
                                       Calibration  0.026 (0.023,0.030)   

Framework                                                                \
Unlabelled OOD Samples                                                    
Method                                                   Oracle [17-19]   
Task                 Year Group        Metric                             
Long LOS             2017 - 2019 [OOD] AUROC        0.705 (0.694,0.715)   
                                       AUPRC        0.554 (0.538,0.571)   
                                       Calibration  0.039 (0.033,0.046)   
Sepsis               2017 - 2019 [OOD] AUROC        0.762 (0.744,0.780)   
                                       AUPRC        0.328 (0.297,0.360)   
                                       Calibration  0.015 (0.011,0.020)   
Mortality            2017 - 2019 [OOD] AUROC        0.917 (0.909,0.925)   
                                       AUPRC        0.521 (0.484,0.556)   
                                       Calibration  0.012 (0.010,0.014)   
Invasive Ventilation 2017 - 2019 [OOD] AUROC        0.882 (0.870,0.894)   
                                       AUPRC        0.609 (0.577,0.642)   
                                       Calibration  0.026 (0.023,0.031)   

Framework                                                                \
Unlabelled OOD Samples                                                    
Method                                                              ERM   
Task                 Year Group        Metric                             
Long LOS             2017 - 2019 [OOD] AUROC        0.696 (0.688,0.704)   
                                       AUPRC        0.559 (0.546,0.572)   
                                       Calibration  0.061 (0.055,0.067)   
Sepsis               2017 - 2019 [OOD] AUROC        0.725 (0.712,0.737)   
                                       AUPRC        0.194 (0.181,0.207)   
                                       Calibration  0.030 (0.026,0.034)   
Mortality            2017 - 2019 [OOD] AUROC        0.898 (0.890,0.906)   
                                       AUPRC        0.554 (0.526,0.581)   
                                       Calibration  0.015 (0.013,0.018)   
Invasive Ventilation 2017 - 2019 [OOD] AUROC        0.886 (0.877,0.895)   
                                       AUPRC        0.635 (0.611,0.658)   
                                       Calibration  0.022 (0.021,0.024)   

Framework                                          Domain Generalization  \
Unlabelled OOD Samples                                                     
Method                                                               IRM   
Task                 Year Group        Metric                              
Long LOS             2017 - 2019 [OOD] AUROC         0.696 (0.688,0.704)   
                 